In [1]:
import sys
!{sys.executable} -m pip install xgboost
print("설치 완료. 커널 재시작 후 코드를 다시 실행해 주세요.")

설치 완료. 커널 재시작 후 코드를 다시 실행해 주세요.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from collections import defaultdict
import time
import os

# ---------------------------------------------------------------------------------
# 1. 파일 경로 및 상수 정의 (실제 파일 경로로 변경 필요)
# ---------------------------------------------------------------------------------
# 🚨 주의: 아래 파일 경로는 실제 환경에 맞게 변경해야 합니다. 
FILE_PATH_MAIN = '[스트롱라이프]최종_데이터_20250924.xlsx'
FILE_PATH_META = "제품 메타데이터 최종.xlsx"
FILE_PATH_ALIGN = "uwellnow_product_align.xlsx" 

ITEM_ID_COL = 'product_id'
PROTEIN_COL = 'protein'
MAX_RANK_COUNT = 7 # 랭킹 데이터는 사용하지 않지만 상수 유지

# OHE Item Features (아이템 특징)
OHE_ITEM_COLS = ['ingredient_type', 'category', 'flavor', 'sensory_tags']
# OHE User Features (사용자 특징)
OHE_USER_COLS = [
    '3) 성별', '8) 운동 활동 기간', '7) 프로틴, 프리워크아웃, 전해질 음료, 게이너 등 헬스 보충제 2종 이상을 섭취해 보신 경험이 있으신가요?', 
    '9) 주에 몇 회 정도 운동을 진행하시나요?(택1)', '10) 알러지 또는 민감성분(복수선택 가능)', '11) 평소 챙기는 끼니는 어떻게 되나요?(복수선택 가능)', 
    '12) 식사 기준으로 운동 시간은 언제인가요?(택 1)', '12-1) 일과(수업,업무,일 등) 기준으로 운동 시간은 언제인가요?(택 1)', 
    '12-2) 운동을 제외한 일과 중 활동은 어느 정도로 활발한가요?(택 1)', '12-3) 시간 기준으로 운동 시작 시간이 언제인가요?(택 1)',
    # Feature Group 2 (영향 요인 및 고려 항목) - LightFM에서 사용했으나, 여기서는 OHE_USER_COLS에 포함하여 활용합니다.
    '13-3) 프로틴의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '13-4) 프로틴의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '13-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '13-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '13-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
    '14-3) 프리워크아웃의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '14-4) 프리워크아웃의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '14-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '14-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '14-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선수택 가능)',
    '15-3) 전해질 음료의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '15-4) 전해질 음료의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '15-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '15-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '15-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
    '16-3) 게이너의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '16-4) 게이너의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '16-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '16-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '16-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
    '17-3) 해당 보충제의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '17-4) 해당 보충제의 효과에서 고려한 점은 무엇인가요?', '17-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '17-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '17-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
]

# 상호작용 컬럼 정의 (재구매 의사 점수를 타겟 변수로 사용)
INTERACTION_WEIGHT_COLS = [
    ('13-9) [프로틴] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)', '13-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)', '13-17) 해당 프로틴에 대한 재구매 의사는 어느 정도인가요?'),
    ('14-9) [프리워크아웃] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)', '14-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)', '14-17) 해당 프리워크아웃에 대한 재구매 의사는 어느 정도인가요?'),
    ('15-9) [전해질 음료(BCAA, 이온음료)] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)', '15-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)', '15-17) 해당 전해질 음료에 대한 재구매 의사는 어느 정도인가요?'),
    ('16-9) [게이너] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)', '16-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)', '16-17) 해당 게이너에 대한 재구매 의사는 어느 정도인가요?'),
    ('17-9) 종합적으로 가장 마음에 들었던 제품 1개를 작성해 주세요', '17-10) 마음에 들었던 제품이 어떤 맛인가요?', '17-17) 해당 제품에 대한 재구매 의사는 어느 정도인가요?'),
]


# ---------------------------------------------------------------------------------
# 2. 필수 데이터 로드 함수 정의 (이전 코드 재사용)
# ---------------------------------------------------------------------------------

def load_and_concatenate_user_data(file_path):
    # ... (함수 내용 유지) ...
    HEADER_ROW_INDEX = 0
    # 파일이 실제로 존재하지 않을 경우를 대비한 Mockup Data
    if not os.path.exists(file_path):
        print(f"경고: 파일 경로 '{file_path}'를 찾을 수 없습니다. Mock Data를 사용합니다.")
        return pd.DataFrame({'user_id': range(1, 10), '3) 성별': ['남', '여'] * 5})[:9].set_index('user_id', drop=False)
        
    df_1st = pd.read_excel(file_path, sheet_name='1차', header=0)
    df_2nd = pd.read_excel(file_path, sheet_name='2차', header=0)
    df_1st.columns = df_1st.columns.astype(str).str.strip()
    df_2nd.columns = df_2nd.columns.astype(str).str.strip()
    col_to_rename = {c: 'no' for c in df_2nd.columns if isinstance(c, str) and c.strip() == 'no.'}
    if col_to_rename:
        df_2nd.rename(columns=col_to_rename, inplace=True)
    df_user_raw = pd.concat([df_1st, df_2nd], ignore_index=True)
    df_user_raw.rename(columns={'no': 'user_id'}, inplace=True)
    return df_user_raw.set_index('user_id', drop=False)


def clean_user_ids(df_user_raw):
    # ... (함수 내용 유지) ...
    df_user_raw.index = df_user_raw.index.to_series().replace('nan', np.nan) 
    valid_user_ids_numeric = pd.to_numeric(df_user_raw.index, errors='coerce')
    valid_indices = df_user_raw.index[valid_user_ids_numeric.notna() & (valid_user_ids_numeric > 0)].unique()
    return df_user_raw.loc[valid_indices].copy()


def normalize_interaction_id(product_name_ac, flavor_ad, mapping_dict):
    """ 사용자 응답 (제품명과 맛)을 딕셔너리 키로 조합하여 정확한 product_id로 정규화합니다. """
    if not product_name_ac or not flavor_ad or not mapping_dict:
        return None 
        
    product_clean = str(product_name_ac).strip().upper()
    flavor_clean = str(flavor_ad).strip().upper()
    search_key = f"{product_clean}_{flavor_clean}"
        
    if search_key in mapping_dict:
        return mapping_dict[search_key]

    combined_name = f"{product_clean}{flavor_clean}"
    return combined_name.replace(' ', '').replace('-', '').replace('.', '').replace('(', '').replace(')', '')


# ---------------------------------------------------------------------------------
# 3. XGBoost를 위한 데이터 전처리 파이프라인
# ---------------------------------------------------------------------------------

def preprocess_for_xgboost(df_user_clean, df_item_raw, df_align):
    
    # 3-1. Item Mapping Data 로드 및 ITEM_FULL_ID_MAP 생성
    df_align.columns = df_align.columns.astype(str).str.strip()
    df_align['MAPPING_KEY'] = (df_align['product'].astype(str).str.strip().str.upper() + 
                                '_' + 
                                df_align['flavor'].astype(str).str.strip().str.upper())

    ITEM_FULL_ID_MAP = pd.Series(
        df_align['product_id'].astype(str).str.strip().str.upper().values,
        index=df_align['MAPPING_KEY']
    ).to_dict()
    
    if 'NAN_NAN' in ITEM_FULL_ID_MAP:
        del ITEM_FULL_ID_MAP['NAN_NAN']


    # 3-2. Interaction Matrix 소스 데이터 구축 (타겟 변수 Y)
    df_interactions_list = []
    
    for item_col, flavor_col, weight_col in INTERACTION_WEIGHT_COLS:
        temp_df = df_user_clean[['user_id', item_col, flavor_col, weight_col]].copy()
        temp_df.rename(columns={item_col: 'product', flavor_col: 'flavor', weight_col: 'weight'}, inplace=True)
        temp_df.dropna(subset=['product', 'flavor'], inplace=True)
        
        # product_id 매핑 및 클리닝
        temp_df['item_id'] = temp_df.apply(lambda row: normalize_interaction_id(row['product'], row['flavor'], ITEM_FULL_ID_MAP), axis=1)
        temp_df.drop(columns=['product', 'flavor'], inplace=True)
        
        # 타겟 변수 (재구매 의사 점수 1-5)
        temp_df['weight'] = pd.to_numeric(temp_df['weight'], errors='coerce')
        # LightFM처럼 증폭하지 않고 1-5점 그대로 사용
        
        df_interactions_list.append(temp_df)
    
    df_interactions = pd.concat(df_interactions_list, ignore_index=True)
    
    df_interactions.columns = df_interactions.columns.astype(str).str.strip().str.lower()
    
    valid_items = df_item_raw[ITEM_ID_COL].unique()
    df_interactions = df_interactions[df_interactions['item_id'].isin(valid_items)]
    df_interactions.dropna(subset=['item_id', 'weight'], inplace=True)
    df_interactions.sort_values(by='weight', ascending=False, inplace=True)
    df_interactions.drop_duplicates(subset=['user_id', 'item_id'], keep='first', inplace=True)
    
    # 3-3. Item Feature 준비 및 One-Hot Encoding
    df_item_raw['protein'] = pd.to_numeric(df_item_raw['protein'], errors='coerce')
    df_item_raw.dropna(subset=['protein', ITEM_ID_COL], inplace=True)
    bins = [0, 15, 25, df_item_raw['protein'].max() + 1]
    labels = ['protein_low', 'protein_mid', 'protein_high']
    df_item_raw['protein_bin'] = pd.cut(df_item_raw['protein'], bins=bins, labels=labels, right=False).astype(str)
    
    selected_ohe_item_cols = [c for c in OHE_ITEM_COLS + ['protein_bin'] if c in df_item_raw.columns]
    
    df_item_features = df_item_raw[[ITEM_ID_COL] + selected_ohe_item_cols].copy()
    
    # 여러 피처 컬럼을 하나의 피처로 통합 (One-Hot Encoding 준비)
    df_item_long = df_item_features.melt(id_vars=[ITEM_ID_COL], value_vars=selected_ohe_item_cols, value_name='feature_value')
    df_item_long.dropna(subset=['feature_value'], inplace=True)
    df_item_long['feature_value'] = df_item_long['feature_value'].astype(str).str.split(r'[,/]')
    df_item_long = df_item_long.explode('feature_value')
    
    df_item_features_ohe = pd.get_dummies(df_item_long, columns=['feature_value'], prefix='item_feat', prefix_sep='_')
    
    # OHE 결과를 Item ID 기준으로 합산하여 Feature Matrix 생성
    item_cols_to_sum = [col for col in df_item_features_ohe.columns if col.startswith('item_feat_')]
    df_item_pivot = df_item_features_ohe.groupby(ITEM_ID_COL)[item_cols_to_sum].sum().reset_index()
    
    # 3-4. User Feature 준비 및 One-Hot Encoding (랭킹 데이터 제외)
    user_ohe_cols_clean = [c for c in OHE_USER_COLS if c in df_user_clean.columns]
    
    df_user_long = df_user_clean[['user_id'] + user_ohe_cols_clean].melt(
        id_vars='user_id', var_name='question', value_name='feature_value'
    ).dropna(subset=['feature_value'])

    df_user_long['feature_value'] = df_user_long['feature_value'].astype(str).str.split(r'[,/]')
    df_user_long = df_user_long.explode('feature_value')
    
    df_user_features_ohe = pd.get_dummies(df_user_long, columns=['feature_value'], prefix='user_feat', prefix_sep='_')

    # OHE 결과를 User ID 기준으로 합산하여 Feature Matrix 생성
    user_cols_to_sum = [col for col in df_user_features_ohe.columns if col.startswith('user_feat_')]
    df_user_pivot = df_user_features_ohe.groupby('user_id')[user_cols_to_sum].sum().reset_index()

    print(f"Item 피처 개수: {len(item_cols_to_sum)}")
    print(f"User 피처 개수: {len(user_cols_to_sum)}")

    # 3-5. Interaction, User, Item Feature 통합 (XGBoost 훈련 데이터셋)
    df_train_data = df_interactions.merge(df_user_pivot, on='user_id', how='left')
    df_train_data = df_train_data.merge(df_item_pivot, on='item_id', how='left')

    # 피처가 없는 행 (Merge 실패 또는 결측치) 제거
    df_train_data.dropna(subset=['weight'], inplace=True)
    df_train_data = df_train_data.fillna(0) # 결측된 피처는 0으로 채움
    
    print(f"XGBoost 최종 학습 데이터 크기: {df_train_data.shape}")
    
    return df_train_data, df_item_raw


# ---------------------------------------------------------------------------------
# 4. 추천 시스템 평가 함수 정의 (RMSE, Precision@K)
# ---------------------------------------------------------------------------------

def precision_at_k_recommender(model, df_test_full, k=3):
    """
    XGBoost 모델의 예측 점수를 사용하여 추천 정밀도(Precision@K)를 계산합니다.
    (협업 필터링 평가 방식을 차용)
    """
    # 1. 테스트 데이터로부터 사용자가 실제로 상호작용한 아이템을 추출
    actual_interactions = defaultdict(set)
    # 🚨 df_test_full은 이미 user_id, item_id, weight를 가지고 있음
    for index, row in df_test_full.iterrows():
        actual_interactions[row['user_id']].add(row['item_id'])
    
    # 2. 모든 가능한 user-item 쌍을 생성하여 예측
    unique_users = df_test_full['user_id'].unique()
    
    # 🚨 예측을 위해 피처만 사용
    X_test_only_features = df_test_full.drop(columns=['user_id', 'item_id', 'weight'])
    
    df_test_full['prediction'] = model.predict(X_test_only_features)
    
    # 3. 사용자별 상위 K개 추천 계산
    total_precision = []
    
    for user in unique_users:
        # 해당 사용자의 모든 예측 점수 가져오기
        user_preds = df_test_full[df_test_full['user_id'] == user].sort_values(
            by='prediction', ascending=False
        )
        
        # 상위 K개 아이템 (예측 점수가 가장 높은 k개)
        top_k_recommended = set(user_preds['item_id'].head(k))
        
        # 실제로 상호작용한 아이템
        user_actual = actual_interactions[user]
        
        # 정밀도 계산: (실제 상호작용 & 추천된 아이템) / K
        if len(top_k_recommended) > 0:
            hits = len(top_k_recommended.intersection(user_actual))
            precision = hits / k
            total_precision.append(precision)
            
    return np.mean(total_precision)


# ---------------------------------------------------------------------------------
# 5. 메인 실행 블록
# ---------------------------------------------------------------------------------

if __name__ == '__main__':
    try:
        start_time = time.time()
        
        # 데이터 로드
        df_user_raw = load_and_concatenate_user_data(FILE_PATH_MAIN)
        df_user_clean = clean_user_ids(df_user_raw)
        
        df_item_raw = pd.read_excel(FILE_PATH_META, sheet_name='제품 메타데이터 최종', header=0)
        df_item_raw.columns = df_item_raw.columns.astype(str).str.strip().str.lower()
        
        df_align = pd.read_excel(FILE_PATH_ALIGN, header=0)
        
        print("✅ 데이터 로드 및 클리닝 완료.")
        
        # XGBoost용 데이터 전처리 및 통합
        df_train_data, df_item_raw_clean = preprocess_for_xgboost(df_user_clean, df_item_raw, df_align)
        
        # XGBoost 학습 데이터셋 분리
        # X 변수에 user_id와 item_id를 남겨서 테스트셋 재구성 시 사용합니다.
        X = df_train_data.drop(columns=['weight']) # weight만 제거
        y = df_train_data['weight'] # 타겟: 재구매 의사 점수 (1-5점)
        
        # 🚨 테스트셋을 분리할 때 user_id와 item_id를 포함합니다. 🚨
        X_train_full, X_test_full, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # 훈련에 사용할 피처(X_train)에서는 user_id, item_id를 제거합니다.
        X_train = X_train_full.drop(columns=['user_id', 'item_id'])
        # 테스트에 사용할 피처(X_test)에서도 user_id, item_id를 제거합니다.
        X_test = X_test_full.drop(columns=['user_id', 'item_id'])

        print(f"\n훈련 데이터 크기: {X_train.shape}, 테스트 데이터 크기: {X_test.shape}")
        
        # ---------------------------------------------------------------------------------
        # 6. XGBoost 모델 학습 (회귀)
        # ---------------------------------------------------------------------------------
        # 희소하고 노이즈가 있는 설문 데이터에 적합하도록 정규화 및 학습 깊이 제어
        
        xgb_model = XGBRegressor(
            n_estimators=100,
            max_depth=5,            # 깊이 제한 (과적합 방지)
            learning_rate=0.1,
            subsample=0.7,          # 데이터 샘플링 (과적합 방지)
            colsample_bytree=0.7,   # 피처 샘플링
            # L2 정규화 
            reg_lambda=1.0,         
            # L1 정규화
            reg_alpha=0.1,          
            random_state=42,
            n_jobs=-1
        )
        
        print("\n▶️ XGBoost Regressor 모델 학습 시작...")
        xgb_model.fit(X_train, y_train)
        print("✅ XGBoost Regressor 모델 학습 완료.")
        
        # ---------------------------------------------------------------------------------
        # 7. 모델 평가
        # ---------------------------------------------------------------------------------
        
        # A. 회귀 지표 평가 (Regression Metrics)
        y_pred = xgb_model.predict(X_test)
        
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        
        # B. 추천 지표 평가 (Recommendation Metrics: Precision@K)
        # 🚨 X_test_full (user_id, item_id, features를 포함)에 예측 결과를 붙여서 사용합니다.
        df_test_full = X_test_full.copy()
        df_test_full['weight'] = y_test

        precision_3 = precision_at_k_recommender(xgb_model, df_test_full, k=3)
        precision_5 = precision_at_k_recommender(xgb_model, df_test_full, k=5)

        end_time = time.time()
        
        print("\n-------------------------------------------------")
        print("                 XGBoost 모델 평가 결과")
        print("-------------------------------------------------")
        print(f"총 실행 시간: {end_time - start_time:.2f}초")
        print("--- 1. 회귀 지표 (Regression Metrics) ---")
        print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
        print(f"MAE (Mean Absolute Error): {mae:.4f}")
        print("\n--- 2. 추천 지표 (Recommendation Metrics) ---")
        print(f"Precision@3: {precision_3:.4f}")
        print(f"Precision@5: {precision_5:.4f}")
        print("-------------------------------------------------")
    
    except FileNotFoundError:
        print("\n❌ 파일이 없습니다: 데이터 파일을 찾을 수 없습니다. 상수의 파일 경로를 확인하거나 Mock Data를 사용하십시오.")
    except Exception as e:
        print(f"\n❌ 최종 모델 구축 중 오류 발생: {e}")


✅ 데이터 로드 및 클리닝 완료.
Item 피처 개수: 134
User 피처 개수: 389

❌ 최종 모델 구축 중 오류 발생: 'item_id'
